In [7]:
import pandas as pd

# Load the data from each spreadsheet
cb = pd.read_excel("data/231013_Customer_Base.xlsx")
ti = pd.read_excel("data/231013_Transactions_Input.xlsx")
fc = pd.read_excel("data/231013_Fraud_Cases.xlsx")

In [8]:
cb.head()

,customer_id,home_bank,customer_prob
0,10298,bank_E,0.49
1,14507,bank_E,0.35
2,14558,bank_E,0.45
3,13453,bank_E,0.63
4,19629,bank_D,0.79


In [9]:
ti.head()

,transaction_id,description,Amount,category,date,month,customer_id,type,In_or_Out,bank_to,bank_from,transac_prob,description_prob,priority
0,1,Salary Deposit - XYZ Corporation,11474.33021,Income,2023-10-01,October,17699,income,paid_in,bank_A,bank_A,NaN,NaN,3
1,2,Online Retailer - Electronics Purchase,199.99000,Online Shopping,2023-10-01,October,10298,spending,paid_out,bank_A,bank_E,0.78,0.25,3
2,3,Utilities Payment - Internet Service,60.00000,Utilities,2023-10-01,October,10298,spending,paid_out,bank_C,bank_E,0.30,0.22,1
3,4,Rent Payment,1500.00000,Housing,2023-10-01,October,14507,spending,paid_out,bank_D,bank_E,0.57,0.53,3
4,5,Paycheck Deposit - Company Name,10480.68449,Income,2023-10-01,October,14507,income,paid_in,bank_E,bank_D,NaN,NaN,3


In [10]:
fc.head()

,transaction_id,is_scam_transaction,fraud_type,case_id
0,187,1,AdvanceFee,5415844
1,247,1,Impersonation,4280670
2,818,1,Impersonation,6956462
3,891,1,Purchase,2702984
4,1142,1,Purchase,3493741


In [11]:
transactions = pd.merge(cb, ti, on='customer_id', how='inner')

In [12]:
transactions.head()

,customer_id,home_bank,customer_prob,transaction_id,description,Amount,category,date,month,type,In_or_Out,bank_to,bank_from,transac_prob,description_prob,priority
0,10298,bank_E,0.49,2,Online Retailer - Electronics Purchase,199.9900,Online Shopping,2023-10-01,October,spending,paid_out,bank_A,bank_E,0.78,0.25,3
1,10298,bank_E,0.49,3,Utilities Payment - Internet Service,60.0000,Utilities,2023-10-01,October,spending,paid_out,bank_C,bank_E,0.30,0.22,1
2,10298,bank_E,0.49,16,Paycheck Deposit - ABC Corp.,10427.6474,Income,2023-10-01,October,income,paid_in,bank_E,bank_E,NaN,NaN,3
3,10298,bank_E,0.49,33,Rent Payment,1200.0000,Housing,2023-10-01,October,spending,paid_out,bank_E,bank_E,0.65,0.62,3
4,10298,bank_E,0.49,2869,Phone Bill Payment - Wireless Provider,70.0000,Utilities,2023-10-02,October,spending,paid_out,bank_A,bank_E,0.48,0.34,1


In [14]:
transactions_out = transactions[transactions['In_or_Out'] == 'paid_in']

In [15]:
transactions_out.head()

,customer_id,home_bank,customer_prob,transaction_id,description,Amount,category,date,month,type,In_or_Out,bank_to,bank_from,transac_prob,description_prob,priority
2,10298,bank_E,0.49,16,Paycheck Deposit - ABC Corp.,10427.6474,Income,2023-10-01,October,income,paid_in,bank_E,bank_E,NaN,NaN,3
15,10298,bank_E,0.49,9312,Transfer from Savings to Current,300.0000,Transfers,2023-10-08,October,income,paid_in,bank_E,bank_E,NaN,NaN,3
28,10298,bank_E,0.49,16449,Transfer from Savings to Current - Emergency Fund,500.0000,Transfers,2023-10-16,October,income,paid_in,bank_E,bank_E,NaN,NaN,3
42,10298,bank_E,0.49,23738,Transfer from Savings to Current,250.0000,Transfers,2023-10-23,October,income,paid_in,bank_E,bank_E,NaN,NaN,3
53,10298,bank_E,0.49,27243,Transfer from Savings to Current,250.0000,Transfers,2023-10-27,October,income,paid_in,bank_E,bank_E,NaN,NaN,3
